In [62]:
import numpy as np
import pandas as pd
import re
import string


In [63]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [64]:
import pickle
with open('../static/model/logistic_regression_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [65]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [66]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [67]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [68]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: " " .join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuation)
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join([ps.stem(x) for x in x.split()]))
    return data["tweet"]

In [69]:
def vectorizer(ds, vocab):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocab))

        for i in range(len(vocab)):
            if vocab[i] in sentence.split():
                sentence_lst[i] = 1
        
        vectorized_lst.append(sentence_lst)
    
    vectorized_lst_new = np.array(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new

In [ ]:
def get_prediction(prediction):
    if prediction == 1:
        return "Positive"
    elif prediction == 0:
        return "Negative"
    else:
        return "Neutral"
    

In [73]:
text = "One of worst products I have ever used. "

preprocessed_text = preprocessing(text)
vectorized_text = vectorizer(preprocessed_text, tokens)
prediction = model.predict(vectorized_text)
get_prediction(prediction[0])

'Positive'